In [ ]:
import numpy as np 
import pandas as pd 
import re

# Data Visualization
import matplotlib.pyplot as plt
%matplotlib inline

import nltk
from nltk.stem import WordNetLemmatizer
import re
from nltk.corpus import stopwords
from keras.preprocessing.text import one_hot
lem=WordNetLemmatizer()


In [ ]:
tweets_data_path = '/content/Corona_NLP_train.csv'
df = pd.read_csv(tweets_data_path, encoding = 'latin-1')
test = pd.read_csv('/content/Corona_NLP_test.csv')

In [ ]:
Y=df.iloc[:,5]
Y=pd.get_dummies(Y)
#Y

In [ ]:
X=df['OriginalTweet']
X=np.array(X)

In [ ]:
nltk.download('wordnet')

In [ ]:
for i,sent in enumerate(X):
  #print(i)
  sent=re.sub('[^a-zA-Z]',' ',sent)
  sent=sent.lower()
  sent=sent.split()
  wds=''
  for word in sent:
    #if word not in stopwords.words('english'):
      word=lem.lemmatize(word)
      wds+=' '
      wds+=word
  X[i]=wds
#X

In [ ]:
from keras.preprocessing.text import Tokenizer
tokenizer=Tokenizer(num_words=100000)
tokenizer.fit_on_texts(X)
dic=tokenizer.word_index

In [ ]:
X_train=tokenizer.texts_to_sequences(X)
#X_train

In [ ]:
from keras.preprocessing.sequence import pad_sequences
X_train=pad_sequences(X_train,maxlen=300,padding='pre')
X_train

In [ ]:
!wget http://nlp.stanford.edu/data/glove.6B.zip

In [ ]:
!unzip glove*.zip

In [ ]:
import os
#embeddings_index = {}
fl = open('/content/glove.6B.300d.txt', encoding='utf-8')
GlvDic={}
for line in fl:
  lst=line.split()
  wrd=lst[0]
  ary=np.array(lst[1:])
  GlvDic[wrd]=ary

In [ ]:
mat=np.zeros((100000,300))
for key,i in dic.items():
  vec=GlvDic.get(key)
  if vec is not None:
    mat[i]=vec

In [ ]:
from keras.models import Sequential
from keras.layers import Embedding
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout
from keras.layers import Bidirectional

In [ ]:
model=Sequential()
model.add(Embedding(100000,300,input_length=300))
model.add(LSTM(100))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.2))

model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(5,activation='softmax'))
model.layers[0].set_weights([mat])
model.layers[0].trainable=False


In [ ]:
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [ ]:
model.fit(X_train,Y,validation_split=.2,epochs=15)
